In [ ]:
# %%capture
!pip install chromadb
!pip install openai==0.28
!pip install environs
!pip install langchain
!pip install tiktoken
!pip install telebot
!pip install pydub
!pip install SpeechRecognition
!pip install langchain-community
!pip install pypdf
!pip install gtts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.

In [ ]:
import openai
from environs import Env
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader, word_document, ImageCaptionLoader
from langchain.document_loaders import PyPDFLoader

In [ ]:
import telebot
import openai

In [ ]:
from pydub import AudioSegment
import speech_recognition

In [ ]:
def ogg_to_wav(audio):
  ogg_audio = AudioSegment.from_ogg(audio)
  return ogg_audio.export("voice.wav", format="wav")

In [ ]:
def stt_func(audio):
  audio = ogg_to_wav(audio)
  recognizer = speech_recognition.Recognizer()
  with speech_recognition.AudioFile(audio) as source:
  # Convert the audio to a recognizable format
    audio = recognizer.record(source)
    text = recognizer.recognize_google(audio,language="ru-RU")
    return text

In [ ]:
loader = PyPDFLoader("Latokent_template (1).pdf")
pages = loader.load()

In [ ]:
embedding = OpenAIEmbeddings(openai_api_key="openai-key")
llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key="openai-key")
print("Beginning")
vectordb = Chroma.from_documents(pages, embedding)
print('Database Loaded')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Beginning
Database Loaded


In [ ]:
template_uz = """Use the context pieces below to answer the question at the end.

Please write all information in Russian. If the user asks about the latoken enterprise, answer it in a longer form.
answer even if the question is not about latoken. Remember, you are bot of LATOKEN

{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT_uz = PromptTemplate(input_variables=["context", "question"], template=template_uz)

# Run chain
qa_chain_uz = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever(), return_source_documents=False,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT_uz})

In [ ]:
def chatbot(question, history=False):
   return str(qa_chain_uz({"query": question})['result'])


In [ ]:
import gtts

def text_to_speech(msg):
    tts = gtts.gTTS(msg, lang='ru')
    tts.save('LATOKEN_AI.mp3')


In [ ]:
import telebot
import openai
import speech_recognition


openai.api_key = "openai-key'


def GPT_chat(prompt):

    completion = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        messages =  [{"role":'user', 'content':prompt}]
    )


    return  completion.choices[0].message.content




TOKEN = "bot-token"
bot = telebot.TeleBot(TOKEN, parse_mode=None)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Здравствуйте, добро пожаловать в бот-помощник Latoken.")

@bot.message_handler(commands=['test'])
def send_welcome(message):
    bot.reply_to(message, "TEST FUNCTION")


@bot.message_handler(content_types=['voice'])
def duck(message):
    file = bot.get_file(message.voice.file_id)
    bytes = bot.download_file(file.file_path)
    with open('voice.ogg', 'wb') as f:
        f.write(bytes)
    text = stt_func("voice.ogg")
    text = chatbot(text)
    text_to_speech(text)
    bot.send_message(message.chat.id, text=text)
    with open('LATOKEN_AI.mp3', 'rb') as f:
        bot.send_audio(message.chat.id, f)



@bot.message_handler(func=lambda message: True)
def echo_all(message):
  sent_message = bot.reply_to(message, text='🚀')
  bot.reply_to(message, GPT_chat(message.text))
  bot.delete_message(chat_id=message.chat.id, message_id=sent_message.message_id)

bot.polling()